In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

# Huvudsidan med kampanjer och topplistor
base_url = 'https://www.adlibris.com/se/kampanj/manadens-topplista'

# Hämta huvudkampanjsidan
response = requests.get(base_url)

# Kontrollera om svaret är OK
if response.status_code == 200:
    # Bearbeta sidan med BeautifulSoup och ange parser
    soup = BeautifulSoup(response.content, 'html.parser')

    # Hitta alla produkter (böcker)
    products = soup.find_all('div', class_='product-panel__product')

    # Lista för att lagra böckerna
    books_data = []

    # Loopa igenom varje bok och extrahera titel, författare och pris
    for product in products:
        # Boktitel
        title = product.find('h3', itemprop='name')
        title_text = title.get_text(strip=True) if title else 'Ingen titel'

        # Författare
        author = product.find('span', itemprop='author')
        author_name = author.get_text(strip=True) if author else 'Ingen författare'

        # Pris
        price = product.find('div', itemprop='offers')
        price_value = price.find
        
        # Lägg till informationen i listan
        books_data.append({
            'title': title_text,
            'author': author_name,
            'price': price_value
        })

    # Skapa en DataFrame från listan
    df = pd.DataFrame(books_data)

    # Skriv ut DataFrame
    print(df)

else:
    print(f"Kunde inte hämta sidan. Statuskod: {response.status_code}")



                                                title  \
0                                        Sömngångaren   
1                         Pappa : berätta om ditt liv   
2                         Mamma : berätta om ditt liv   
3   Fuskbygget : så knäckte bostadsmarknaden Sveri...   
4                                       På Julbordsön   
5                    Väninnorna på Nordiska Kompaniet   
6             Läs : boken för dig som vill börja läsa   
7                         Guinness World Records 2025   
8                   Pixi adventskalender Emma Göthner   
9                        Saker som (nästan) ingen vet   
10                         Sommarskuggan och julbuset   
11                             Trollbergets hemlighet   
12                               Bara rumpor på julen   
13                     Tur och retur : min berättelse   
14         Läs pysselbok : öva på att läsa och skriva   
15                        A Court of Thorns and Roses   
16                       Mormor

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   49 non-null     object
 1   author  49 non-null     object
 2   price   49 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [3]:
import sqlite3
# Skapa en anslutning till databasen
con = sqlite3.connect('bok.db')

# Kontrollera att DataFrame inte har några felaktiga datatyper
df = df.map(str)  # Om du misstänker att det finns metoder eller obehöriga datatyper

# Skriv DataFrame till SQLite-databasen, ersätt om tabellen redan finns
df.to_sql('bokdata', con, if_exists='replace', index=False)

49

In [4]:
# Konvertera 'author' till str
df['author'] = df['author'].astype(str)

In [5]:
# Bekräfta datatyper
print(df.dtypes)

title     object
author    object
price     object
dtype: object


In [6]:
print(type(df['author'][0]))

<class 'str'>


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   49 non-null     object
 1   author  49 non-null     object
 2   price   49 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [8]:
# Kontrollera om alla värden är av typen str
print(all(isinstance(x, str) for x in df['author']))

True


In [9]:
# Konvertera till pandas nya string-datatyp
df['author'] = df['author'].astype('string')

In [12]:
# Bekräfta datatyper
print(df.dtypes)

title             object
author    string[python]
price             object
dtype: object


In [11]:
print(df.head())  # Visa de första 5 raderna
print(df.info())  # Visa information om kolumner och datatyper
print(df.describe())  # Visa en beskrivning

                                               title            author  \
0                                       Sömngångaren       Lars Kepler   
1                        Pappa : berätta om ditt liv  Ingen författare   
2                        Mamma : berätta om ditt liv  Ingen författare   
3  Fuskbygget : så knäckte bostadsmarknaden Sveri...  Andreas Cervenka   
4                                      På Julbordsön              Tomu   

                                               price  
0  <bound method Tag.find of <div class="price se...  
1  <bound method Tag.find of <div class="price se...  
2  <bound method Tag.find of <div class="price se...  
3  <bound method Tag.find of <div class="price se...  
4  <bound method Tag.find of <div class="price se...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   49 non-null     object
 1   author  49

In [13]:
df.drop(columns=['price'], inplace=True)
print(df)

                                                title  \
0                                        Sömngångaren   
1                         Pappa : berätta om ditt liv   
2                         Mamma : berätta om ditt liv   
3   Fuskbygget : så knäckte bostadsmarknaden Sveri...   
4                                       På Julbordsön   
5                    Väninnorna på Nordiska Kompaniet   
6             Läs : boken för dig som vill börja läsa   
7                         Guinness World Records 2025   
8                   Pixi adventskalender Emma Göthner   
9                        Saker som (nästan) ingen vet   
10                         Sommarskuggan och julbuset   
11                             Trollbergets hemlighet   
12                               Bara rumpor på julen   
13                     Tur och retur : min berättelse   
14         Läs pysselbok : öva på att läsa och skriva   
15                        A Court of Thorns and Roses   
16                       Mormor